In [1]:
from scgenome.simulation import many_poisson_bicluster

TRIALS_PER_SET = 100
SAMPLES_PER_CLUSTER = [4]
NUM_BIN = [50, 100]
MAX_CN = [4, 8]
ALPHA = [0.1, 0.3]
INIT_LAMBDAS = [(3,1)]
JUMP_LAMBDAS = [(1,0.1)]

sims = many_poisson_bicluster(TRIALS_PER_SET, SAMPLES_PER_CLUSTER, NUM_BIN, MAX_CN, ALPHA, INIT_LAMBDAS, JUMP_LAMBDAS)
display(sims.head())

/Users/massoudmaher/Documents/Code/scgenome/scgenome/simulation.py:208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  plinkage["r_merge"] = plinkage["r_merge"].astype("float")
/Users/massoudmaher/Documents/Code/scgenome/scgenome/simulation.py:209: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  plinkage["dist"] = -1 * plinkage["r_merge"]


,samples_per_cluster,num_bin,max_cn,alpha,init_lambdas,jump_lambdas,cn_data,plinkage,plot_data,clustering,prop_correct
0,4,50,4,0.1,"(3, 1)","(1, 0.1)",chr bin cell_id state start end clu...,i j r_merge merge_count dis...,"[[4.0, 5.0, 215.48938266598992, 2.0], [8.0, 6....",sample_inds cell_id exp_cl obs_cl 0 ...,0.750
1,4,50,4,0.3,"(3, 1)","(1, 0.1)",chr bin cell_id state start end clu...,i j r_merge merge_count dis...,"[[5.0, 7.0, 185.51764387515115, 2.0], [4.0, 8....",sample_inds cell_id exp_cl obs_cl 0 ...,0.875
2,4,50,8,0.1,"(3, 1)","(1, 0.1)",chr bin cell_id state start end clu...,i j r_merge merge_count d...,"[[5.0, 6.0, 327.36555037373216, 2.0], [4.0, 8....",sample_inds cell_id exp_cl obs_cl 0 ...,0.625
3,4,50,8,0.3,"(3, 1)","(1, 0.1)",chr bin cell_id state start end clu...,i j r_merge merge_count d...,"[[0.0, 4.0, 964.4876757451372, 2.0], [5.0, 7.0...",sample_inds cell_id exp_cl obs_cl 0 ...,0.625
4,4,100,4,0.1,"(3, 1)","(1, 0.1)",chr bin cell_id state start end clu...,i j r_merge merge_count d...,"[[5.0, 6.0, 827.7760596799748, 2.0], [4.0, 7.0...",sample_inds cell_id exp_cl obs_cl 0 ...,0.625


In [3]:
#sims.reset_index(drop=True).to_json("~/Desktop/sims.json")
#import pandas as pd
#csims = pd.read_json("~/Desktop/sims.json")
#pd.DataFrame(csims.iloc[0,6])

In [8]:
sims.dtypes

samples_per_cluster      int64
num_bin                  int64
max_cn                   int64
alpha                  float64
init_lambdas            object
jump_lambdas            object
cn_data                 object
plinkage                object
plot_data               object
clustering              object
prop_correct           float64
dtype: object